# Binary Classification Algorithm for Identifying Calisthenics Spots in Images

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install Requirements

In [5]:
!pip install openimages numpy pandas scikit-learn opencv-python bing-image-downloader

## Imports

In [6]:
import os
import random
import cv2
import numpy as np
# from openimages.download import download_images
from bing_image_downloader import downloader

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

## Utility Functions

In [7]:
def load_images_from_folder(folder_path, num_images):
    images = []
    random_files = random.sample(os.listdir(folder_path), num_images)
    for filename in random_files:
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

def test_model(model, image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))  # Resize the image to match the input size during training
        img = img.reshape(1, -1) / 255.0  # Flatten and normalize the image
        prediction = model.predict(img)
        if prediction[0] == 1:
            return "calisthenics_spot"
        else:
            return "not_calisthenics_spot"
    else:
        return "Invalid image path or format"


def resize_images_by_percentage(image_folder, percentage, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    resized_images = []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            new_width = int(img.shape[1] - (img.shape[1] * percentage))
            new_height = int(img.shape[0] - (img.shape[0] * percentage))
            resized_img = cv2.resize(img, (new_width, new_height))
            resized_images.append(resized_img)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_img)

    return np.array(resized_images)

## Resize calisthenics spots

In [12]:
# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot"
not_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/random_images_dataset"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# Specify the percentage by which to resize the images
resize_percentage = 0.8095238

# Resize calisthenics_spot images by the specified percentage
resized_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/resized_calisthenics_spotsss"
resized_calisthenics_images = resize_images_by_percentage(calisthenics_folder, resize_percentage, resized_calisthenics_folder)

# # Download random images from the Open Images V6 dataset for the not_calisthenics_spot class
num_not_calisthenics_images = 6552
# class_label = "not_calisthenics_spot"
# # download_images(limit=num_not_calisthenics_images, annotations=class_label,
# #                 folder=not_calisthenics_folder)
# downloader.download(query='', limit=num_not_calisthenics_images, output_dir=not_calisthenics_folder, adult_filter_off=False, force_replace=False, timeout=60, verbose=True)

# Load images from the not_calisthenics_spot folder
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

<ipython-input-9-eb366fed2886>:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(resized_images)


## Load images from folder


In [8]:
# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/resized_calisthenics_spotsss"
not_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/random_images_dataset"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# Load images from the not calisthenics_spot folder
num_not_calisthenics_images = 6552
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

KeyboardInterrupt: ignored

## Train the model

In [20]:
# Create labels for the data
calisthenics_labels = np.ones(shape=num_calisthenics_images, dtype=int)
not_calisthenics_labels = np.zeros(shape=num_not_calisthenics_images, dtype=int)

# Combine the data and labels for the model
X = np.array(object=(calisthenics_images + not_calisthenics_images), dtype=object)
y = np.concatenate((calisthenics_labels, not_calisthenics_labels))

# Resize the images to a consistent size (e.g., 64x64)
resized_images = [cv2.resize(image, (64, 64)) for image in X]

# Normalize the pixel values of each image
normalized_images = [image.astype('float32') / 255.0 for image in resized_images]

# Convert the list of images to a NumPy array
X_normalized = np.array(normalized_images)

# Reshape the images to match the flattened size
num_samples, height, width, channels = X_normalized.shape
X_flattened = X_normalized.reshape(num_samples, -1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_flattened, y, test_size=0.2, random_state=42)

# Create and train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6921022510492179


## Imports

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

NameError: ignored

## Save the model

In [ ]:
# Save the trained SVM model
model_filename = '/content/drive/Othercomputers/Meu computador/5-periodo/method/svm_model_1.pkl'
joblib.dump(svm_classifier, model_filename)

## Load and test the model

In [ ]:
# Load the trained SVM model
loaded_model = joblib.load(model_filename)

def test_model(model, image_path):
    img = cv2.imread(image_path)
    if img is None:
        print("Failed to load the image.")
        return

    # Preprocess the image: resize and normalize
    resized_img = cv2.resize(img, (64, 64))
    normalized_img = resized_img.astype('float32') / 255.0
    flattened_img = normalized_img.reshape(1, -1)  # Reshape for prediction

    # Make predictions using the loaded model
    prediction = model.predict(flattened_img)

    if prediction[0] == 1:
        print("The image is a calisthenics_spot.")
    else:
        print("The image is not a calisthenics_spot.")

# Path to the image you want to test
test_image_path = "/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot/teste/epa.png"  # Replace with your test image path

# Test the model using the loaded SVM model
test_model(loaded_model, test_image_path)